I wanted to see if using deep learning could improve my results.  I decided to try out a bi-directional LSTM with a CRF layer in PyTorch.

In [ ]:
import pickle
with open('restaurant_iob.pickle', 'rb') as handle:
    data = pickle.load(handle)

Get all unique tags and all unique words

In [ ]:
tag_set = set([x for y in [set([x['iob_tag']for x in sentence]) for sentence in data] for x in y])
word_set = set([x for y in [set([x['text']for x in sentence]) for sentence in data] for x in y])

As the models requires them to be integers, convert them

In [ ]:
from future.utils import iteritems

word2idx = {w: i for i, w in enumerate(word_set)}
tag2idx = {t: i for i, t in enumerate(tag_set)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

Pad sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

maxlen = max([len(s) for s in data])
print(maxlen)

79


In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
# Create numerical values for tags
tag_to_ix = {"O": 0, "I": 1, "B": 2, START_TAG: 3, STOP_TAG: 4}

In [ ]:
X = [[token['text'] for token in sentence] for sentence in data]
#X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=len(word_set) - 1)
y = [[token['iob_tag'] for token in sentence] for sentence in data]
#y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag_to_ix["O"])

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train[0]

['However',
 ',',
 'I',
 'do',
 'not',
 'understand',
 'the',
 'extraordinary',
 'hype',
 'about',
 'this',
 'restaurant',
 '.']

In [ ]:
y_train[0]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm as progress_bar
torch.manual_seed(1)

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
training_data = list(zip(X_train, y_train))

In [ ]:
model = BiLSTM_CRF(len(word2idx), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in progress_bar(range(50)):  # takes about 45 minutes to run on colab
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word2idx)
        
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
      
        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word2idx)
    
    print(model(precheck_sent))
# We got it!

100%|██████████| 50/50 [45:36<00:00, 54.74s/it]

(tensor(106.3559), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
# Create numerical values for tags
ix_to_tag = {"0": "O", "1": "I", "2": "B", "3": START_TAG, 4: STOP_TAG}

In [ ]:
test_data = list(zip(X_test, y_test))
y_pred = []
for i in range(len(y_test)):
    precheck_sent = prepare_sequence(test_data[i][0], word2idx)
    pred = [ix_to_tag[str(x)] for x in model(precheck_sent)[1]]
    y_pred.append(pred)

In [ ]:
len(X_test)

609

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=['B', 'I'])

0.6918232421621968

In [ ]:
#!pip install sklearn_crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(
    y_test, y_pred, labels=['B', 'I'], digits=3
))

              precision    recall  f1-score   support

           B      0.744     0.771     0.757       743
           I      0.526     0.524     0.525       292

   micro avg      0.684     0.701     0.693      1035
   macro avg      0.635     0.648     0.641      1035
weighted avg      0.683     0.701     0.692      1035



The results are slightly worse than the original CRF model which was trained to fit the specific words.  Recall is almost identical, but precision is a little higher leading me to wonder if the model here is making predictions on a similar basis but it just experiencing a higher number of false positives.

We can explore the specific predictions in more detail.

In [ ]:
import pandas as pd
pred_table = []

for sentence, labels, predictions in zip(X_test, y_test, y_pred):
    for word, label, prediction in zip(sentence, labels, predictions):
        pred_table.append((word, label, prediction))

pred_df = pd.DataFrame(pred_table, columns = ['token', 'label', 'pred'])
pred_df

,token,label,pred
0,The,O,O
1,ambience,B,B
2,is,O,O
3,authentic,O,O
4,and,O,O
...,...,...,...
9744,I,O,O
9745,would,O,O
9746,go,O,O
9747,again,O,O


In [ ]:
pred_df.to_csv("bilstmcrf_50epochs.csv", index = False)

I think the next step here is to see if we can incorporate pre-trained word embeddings in the hope that it'll improve performance on new unseen words, and also to see if perhaps fitting a deep learning model to the features without the words themselves can prove at all useful.